In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score


In [6]:
#pip install torch torchvision torchaudio

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/96/4e/970cd3e13ad95aed81102272f0678d8cc48101880b8be5bae8aad22e7f3b/torch-2.2.0-cp311-none-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/3e/4f/ad5c2a7d2783649c8ea691441a9f285accae922a1625e21603c45e3ddff4/torchvision-0.17.0-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for torchaudio from https://files.pythonhosted.org/packages/79/f7/5929802a1d14693d2dea6e60c51a923724348f134a91558f22bc686d3d8b/torchaudio-2.2.0-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for typing-extensions>=4.8.0 from https://files.pythonhosted.org/packages/b7/f4/6a90020cd2d93349b442bfcb657d0dc91eee65491600b2cb1d388bc98e6b/typing_extensions-4.9.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 5.7 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from torch import nn
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
from torchvision import datasets
from torch.utils.data import TensorDataset

In [3]:
from sklearn.preprocessing import StandardScaler

In [4]:
data = pd.read_csv("card_transdata.csv")

In [5]:
# Separate features and target variable and generate the training and test sets
X_train = data.iloc[:500000,:7]
X_test = data.iloc[500000:,:7]
y_train = data.iloc[:500000,-1]
y_test = data.iloc[500000:,-1]

In [6]:
#convert the data into a PyTorch Dataset class

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)
# Update your Dataset and DataLoader accordingly
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [8]:
#Define the Neural Network Architecture

# Experiment with different batch sizes
batch_size = 64 

class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()  # This might be unnecessary for already flattened input
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, hidden_size),  # Adjusted input size
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_classes),
        )

    def forward(self, x):
        x = self.flatten(x)  # Consider removing if input is already flat
        logits = self.linear_relu_stack(x)
        return logits

# Adjust the model instantiation accordingly
input_size = 7  # Adjusted for your dataset
num_classes = 10  # Assuming this is correct for your task


In [9]:
#Optimization Loop, here we can experiment with the parameters
learning_rate = 1e-3
batch_size = 64
epochs = 5
# Hyperparameters
hyperparams = {
    'input_size': X_train.shape[1],
    'hidden_size': 64,  # we can experiment with this
    'batch_size': 64,   # we can experiment with this
    'learning_rate': 0.01,  # we can experiment with this
    'num_epochs': 10   # we can experiment with this
}

In [10]:
train_dataloader = DataLoader(train_dataset, batch_size=hyperparams["batch_size"], shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=hyperparams["batch_size"], shuffle=False)
model = NeuralNetwork(hyperparams["input_size"], hyperparams["hidden_size"], num_classes)

In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=hyperparams["learning_rate"])

In [12]:
#Full implementation
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [13]:
#Training the Models
epochs = 10 #here we try to improve the neuro network model
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 3.197701  [   64/500000]
loss: 0.584439  [ 6464/500000]
loss: 0.515197  [12864/500000]
loss: 0.521941  [19264/500000]
loss: 0.311586  [25664/500000]
loss: 0.542169  [32064/500000]
loss: 0.226477  [38464/500000]
loss: 0.414987  [44864/500000]
loss: 0.269992  [51264/500000]
loss: 0.264687  [57664/500000]
loss: 0.174859  [64064/500000]
loss: 0.337666  [70464/500000]
loss: 1.085082  [76864/500000]
loss: 0.147570  [83264/500000]
loss: 0.288419  [89664/500000]
loss: 0.311441  [96064/500000]
loss: 0.308966  [102464/500000]
loss: 0.147722  [108864/500000]
loss: 0.179376  [115264/500000]
loss: 0.121280  [121664/500000]
loss: 0.257264  [128064/500000]
loss: 0.228659  [134464/500000]
loss: 0.267845  [140864/500000]
loss: 0.064499  [147264/500000]
loss: 0.092105  [153664/500000]
loss: 0.184127  [160064/500000]
loss: 0.159294  [166464/500000]
loss: 0.151893  [172864/500000]
loss: 0.099691  [179264/500000]
loss: 0.388206  [185664/500000]
loss: 0.112199  

loss: 0.072709  [76864/500000]
loss: 0.097480  [83264/500000]
loss: 0.091192  [89664/500000]
loss: 0.035251  [96064/500000]
loss: 0.054417  [102464/500000]
loss: 0.090423  [108864/500000]
loss: 0.022432  [115264/500000]
loss: 0.066504  [121664/500000]
loss: 0.151831  [128064/500000]
loss: 0.081101  [134464/500000]
loss: 0.040976  [140864/500000]
loss: 0.067793  [147264/500000]
loss: 0.115658  [153664/500000]
loss: 0.058660  [160064/500000]
loss: 0.076652  [166464/500000]
loss: 0.059741  [172864/500000]
loss: 0.090508  [179264/500000]
loss: 0.061917  [185664/500000]
loss: 0.112633  [192064/500000]
loss: 0.151178  [198464/500000]
loss: 0.047659  [204864/500000]
loss: 0.053396  [211264/500000]
loss: 0.069363  [217664/500000]
loss: 0.026033  [224064/500000]
loss: 0.070944  [230464/500000]
loss: 0.053260  [236864/500000]
loss: 0.076281  [243264/500000]
loss: 0.049538  [249664/500000]
loss: 0.084538  [256064/500000]
loss: 0.060888  [262464/500000]
loss: 0.022050  [268864/500000]
loss: 0.0589

loss: 0.048557  [166464/500000]
loss: 0.044173  [172864/500000]
loss: 0.071646  [179264/500000]
loss: 0.005184  [185664/500000]
loss: 0.037637  [192064/500000]
loss: 0.032813  [198464/500000]
loss: 0.082979  [204864/500000]
loss: 0.052850  [211264/500000]
loss: 0.098852  [217664/500000]
loss: 0.041914  [224064/500000]
loss: 0.034572  [230464/500000]
loss: 0.047776  [236864/500000]
loss: 0.018633  [243264/500000]
loss: 0.069881  [249664/500000]
loss: 0.049926  [256064/500000]
loss: 0.026579  [262464/500000]
loss: 0.019739  [268864/500000]
loss: 0.052603  [275264/500000]
loss: 0.131870  [281664/500000]
loss: 0.033346  [288064/500000]
loss: 0.092484  [294464/500000]
loss: 0.037806  [300864/500000]
loss: 0.050001  [307264/500000]
loss: 0.065598  [313664/500000]
loss: 0.043827  [320064/500000]
loss: 0.027051  [326464/500000]
loss: 0.038352  [332864/500000]
loss: 0.045273  [339264/500000]
loss: 0.053637  [345664/500000]
loss: 0.076132  [352064/500000]
loss: 0.070247  [358464/500000]
loss: 0.

loss: 0.035704  [262464/500000]
loss: 0.005124  [268864/500000]
loss: 0.049581  [275264/500000]
loss: 0.019751  [281664/500000]
loss: 0.022408  [288064/500000]
loss: 0.055038  [294464/500000]
loss: 0.040868  [300864/500000]
loss: 0.018371  [307264/500000]
loss: 0.086306  [313664/500000]
loss: 0.009909  [320064/500000]
loss: 0.044370  [326464/500000]
loss: 0.029228  [332864/500000]
loss: 0.107885  [339264/500000]
loss: 0.080353  [345664/500000]
loss: 0.038020  [352064/500000]
loss: 0.043784  [358464/500000]
loss: 0.037741  [364864/500000]
loss: 0.016938  [371264/500000]
loss: 0.020479  [377664/500000]
loss: 0.039307  [384064/500000]
loss: 0.079852  [390464/500000]
loss: 0.028238  [396864/500000]
loss: 0.039414  [403264/500000]
loss: 0.087225  [409664/500000]
loss: 0.006330  [416064/500000]
loss: 0.025561  [422464/500000]
loss: 0.069156  [428864/500000]
loss: 0.037948  [435264/500000]
loss: 0.022846  [441664/500000]
loss: 0.042681  [448064/500000]
loss: 0.045717  [454464/500000]
loss: 0.

In [14]:
#Evaluate the Model
model.eval()  # Ensure the model is in evaluation mode
all_preds = []
all_targets = []

with torch.no_grad():
    for X, y in test_dataloader:
        pred = model(X)
        all_preds.extend(pred.argmax(dim=1).cpu().numpy())  # Collect predictions
        all_targets.extend(y.cpu().numpy())  # Collect true labels

# Calculate F1 Score
f1 = f1_score(all_targets, all_preds, average='weighted')
print(f"F1 Score: {f1:.4f}")

# Calculate Accuracy
accuracy = accuracy_score(all_targets, all_preds)
print(f"Accuracy: {accuracy:.4f}")


F1 Score: 0.9757
Accuracy: 0.9749


In [15]:
#according to last homework, a single decision tree model produces the F1 score 0.9998, while here we can
#produce F1 score 0.98. Therefore a simple decision tree is better than the neural network model here.